In [ ]:
import pandas as pd
import numpy as np
import os
import sqlite3
import re

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Pre-Process Data for Database
In this notebook, we pre-process the three primary tables we plan to utilize for our application:
1. Album of The Year critic ratings
- Features user and critic review aggregated by Album of the Year and Metacritic
- The dataset includes more than 30K rows of albums and aggregated critic and user review scores
- Link to source https://www.kaggle.com/kauvinlucas/30000-albums-aggregated-review-ratings

2. Spotify API
- More than 1.2M songs collected from Spotify’s API
- The dataset includes features like name, artists, album, release date, key, explicit flag, duration, popularity, danceability, and more
- The data was created in December 2020. Each row represents a single track
- Link to source https://www.kaggle.com/rodolfofigueroa/spotify-12m-songs

3. Pitchfork reviews
- Includes over 18K Pitchfork (an online music magazine) review going back to January 1999
- The database contains separate tables on artists, content, genres, labels, reviews, and years
- Link to source https://www.kaggle.com/nolanbconaway/pitchfork-data

The notebook will read in the datasets, process, and output the final datasets in csv that will ultimately be uploaded to AWS RDS.

# Read the datasets

## 1. AOTY Ratings

In [ ]:
# Read
df_ratings = pd.read_csv('./album_ratings.csv') # Metacritic/AOTY

print(df_ratings.shape)
df_ratings.head(2)

## 2. Spotify API

In [ ]:
# Read
df_spotify = pd.read_csv('./tracks_features.csv') #Spotify/Kaggle 1.2M tracks
print(df_spotify.shape)
df_spotify.head(2)

## 3. Pitchfork reviews

In [ ]:
# Read sqlite query results into a pandas DataFrame
con = sqlite3.connect("./database.sqlite")

# Read
query_str = \
"""
SELECT 
    t1.reviewid
    , t1.title
    , t1.artist
    , t1.url
    , t1.score
    , t1.author
    , t1.pub_date
    , t1.best_new_music
    , t2.content
    
FROM reviews t1 JOIN content t2 ON t1.reviewid = t2.reviewid
"""

df_pitchfork = pd.read_sql_query(query_str, con)
con.close()

print(df_pitchfork.shape)
df_pitchfork.head(2)

# Pre-process

#### All datasets: Select only the columns we need, rename fields, and create a lowercased version of the album and artist
The lowercased album and artist will help joins across the three datasets.

In [ ]:
# AOTY
df_ratings = df_ratings[[
    'Artist'
    , 'Title'
    , 'Format'
    , 'Label'
    , 'Genre'
    , 'AOTY Critic Score'
    , 'AOTY Critic Reviews'
    , 'AOTY User Score'
    , 'AOTY User Reviews'  
]].copy()
df_ratings = df_ratings.rename(columns={"Title": "album_name", "Artist": "artist"})
df_ratings['album_name_lc'] = df_ratings['album_name'].str.lower()
df_ratings['artist_lc'] = df_ratings['artist'].str.lower()

# Spotify
# Select all columns for this dataset
df_spotify = df_spotify.rename(
                            columns={"id": "track_id"
                                     , "name": "track_name"
                                     , "album": "album_name"
                                     , "artists": "artist"
                                    })
df_spotify['album_name_lc'] = df_spotify['album_name'].str.lower()
df_spotify['artist_lc'] = df_spotify['artist'].str.lower()

# Pitchfork
df_pitchfork = df_pitchfork.rename(
                                columns={
                                    "title": "album_name",
                                    "url": "pf_url",
                                    "score": "pf_score",
                                    "author": "pf_author",
                                    "pub_date": "pf_pubdate",
                                    "content": "pf_review"
                                })
df_pitchfork['album_name_lc'] = df_pitchfork['album_name'].str.lower()
df_pitchfork['artist_lc'] = df_pitchfork['artist'].str.lower()

#### Spotify API: Parse out the first artist

In [ ]:
# Function for cleaning Spotify artists
def getFirstArtist(s):
    '''
    This function removes the list like structure of the "artists" field in the Spotify dataset. Pick the first artist in the list
    
    Input : List of artists (str)
    Output: One artist (str)
    '''
    s = s.replace("[", "").replace("]", "")
    first_element = s.find(",")
    return s[:first_element].replace("'", "")

# Clean
df_spotify['artist'] = df_spotify['artist'].apply(lambda s:getFirstArtist(s))
df_spotify['artist_lc'] = df_spotify['artist_lc'].apply(lambda s:getFirstArtist(s))
df_spotify['artist_ids'] = df_spotify['artist_ids'].apply(lambda s:getFirstArtist(s))

df_spotify.head()

#### Pitchfork Reviews: Clean the album names so that future joins are more likely to match across the three datasets

In [ ]:
# Function for cleaning Pitchfork album names
def cleanPitchforkAlbumNames(s):
    '''
    This function sanitizes the album names in the Pitchfork dataset. The album names are rather specific, and will include details
    like "40th anniversary edition". So let's remove that
    
    Input : A dirty album name (str)
    Output: A clean album name (str)
    '''
    
    # Remove anything in brackets. Usually this will denote the edition
    bracket_check = s.find("[")
    if bracket_check != -1:
        s = s[:bracket_check].strip()
        
    # Remove anything that comes after a "n"th annivesary edition using regex
    nth_anniv_check = re.search(r'[0-9]+(th|st|nd|rd)', s)
    if nth_anniv_check:
        ix = nth_anniv_check.start()
        s = s[:ix].strip(": ")
    
    return s

# Clean
df_pitchfork['orig_album_name'] = df_pitchfork['album_name']
df_pitchfork['album_name'] = df_pitchfork['album_name'].apply(lambda s:cleanPitchforkAlbumNames(s))

# Check that PF album sanitation worked
# 'album_name' is the sanitized field
df_pitchfork.sort_values('pf_score', ascending = False)[['orig_album_name', 'album_name']][:10]

# General EDA
How many albums overlap between the three datasets?

In [ ]:
# Subset to only albums and artists
t1 = df_ratings[['album_name_lc', 'artist_lc']].drop_duplicates()
t2 = df_spotify[['album_name_lc', 'artist_lc']].drop_duplicates()
t3 = df_pitchfork[['album_name_lc', 'artist_lc']].drop_duplicates()

# How many unique albums are there in each dataset?
# Albums with same name but different artist are counted as different
print('How many unique albums are there in each dataset?')
print('AOTY: ', len(t1))
print('Spotify: ', len(t2))
print('Pitchfork: ', len(t3))

## 4. Genres

In [ ]:
df_genres = pd.read_csv('./genres-revised.csv')
df_genres.head()

#### Genres: Add Ids to Ratings

In [ ]:
# Functions to transform genres file and apply to ratings
def rename_genres(genre):
    for column in df_genres:
        if genre in df_genres[column].values:
            return column

def genre_index(genre):
    if genre in df_genres.columns:
        return df_genres.columns.get_loc(genre)

# narrow genre options for the ratings data
df_ratings['Genre'] = df_ratings['Genre'].apply(lambda d: rename_genres(d))
df_ratings['genre_id'] = df_ratings['Genre'].apply(lambda d: genre_index(d))

df_ratings['genre_id'] = pd.to_numeric(df_ratings['genre_id'], downcast='integer')
df_genre_id_column = pd.DataFrame(
    {
        "genre_id": pd.Series(df_ratings['genre_id'], dtype=np.dtype("int64"))
    }
)
df_genre_id_column = df_genre_id_column.convert_dtypes()
df_ratings['genre_id'] = df_genre_id_column['genre_id']

df_ratings.head()

## 5. Labels

In [ ]:
df_labels = pd.DataFrame({"Label": df_ratings['Label'].drop_duplicates()})
df_labels.head()

In [ ]:
def label_index(label):
    for row in df_labels['Label']:
        if row == label:
            return df_labels.index[df_labels['Label'] == label][0]

#### Labels: Add Ids to Ratings

In [ ]:
df_ratings['label_id'] = df_ratings['Label'].apply(lambda d: label_index(d))

df_label_id_column = pd.DataFrame(
    {
        "label_id": pd.Series(df_ratings['label_id'], dtype=np.dtype("int64"))
    }
)
df_label_id_column = df_label_id_column.convert_dtypes()
df_ratings['label_id'] = df_label_id_column['label_id']

df_ratings.head()

## Create Dataset of All But Pitchfork

In [ ]:
df_merged = pd.merge(df_spotify, df_ratings, on=['artist_lc', 'album_name_lc'])
df_merged.head()

### Create .csv Files

In [ ]:
# Clean master dataset before creating each of the normalized tables
df_culled = df_merged.drop(columns=['album_name_x', 'artist_x', 'release_date']) \
            .rename(columns={"album_name_y": "album_name","artist_y": "artist"})

df_culled.shape

In [ ]:
# Need to dedup albums, as some have multiple versions (explicit and clean)
df_album = df_culled[[
    'album_id'
    , 'album_name'
    , 'artist_ids'
    , 'genre_id'
    , 'label_id'
    , 'year'
    , 'Format'
    , 'AOTY Critic Score'
    , 'AOTY Critic Reviews'
    , 'AOTY User Score'
    , 'AOTY User Reviews']].drop_duplicates(subset = ['album_name', 'artist_ids'], keep = 'first')

# Update master dataset by filtering for the deduped albums
df_culled = df_culled[df_culled['album_id'].isin(df_album['album_id'])].copy()

In [ ]:
# Pull out and save seed tables
df_album.to_csv('./seed/album.csv', index = False)
df_culled[['artist', 'artist_ids']].drop_duplicates().to_csv('./seed/artist.csv', index = False)
df_culled[['Genre', 'genre_id']].dropna().drop_duplicates().to_csv('./seed/genre.csv', index = False)
df_culled[['Label', 'label_id']].dropna().drop_duplicates().to_csv('./seed/label.csv', index = False)
df_culled[[
    'track_id', 'track_name', 'album_id', 'disc_number'
    , 'track_number', 'danceability', 'energy', 'key', 'loudness'
    , 'mode', 'speechiness', 'acousticness', 'instrumentalness'
    , 'liveness', 'valence', 'tempo', 'explicit', 'duration_ms', 'time_signature']] \
    .drop_duplicates().to_csv('./seed/song.csv', index = False)

# Create Reviews and Authors (from Pitchfork)

In [ ]:
df_culled2 = pd.merge(
    df_culled.drop_duplicates(subset = ['album_name', 'artist_ids'], keep = 'first')
    , df_pitchfork
    , on=['artist_lc', 'album_name_lc']
    , how = 'inner')
df_culled2.shape

In [ ]:
# Save seed table
df_culled2 = df_culled2.reset_index().rename(columns = {'index': 'id'})
df_culled2[['id', 'album_id', 'pf_url', 'pf_score', 'pf_pubdate']].to_csv('./review.csv', index = False)